In [7]:
import numpy as np
import pandas as pd

class Datagenerater:
    def __init__(self):
        self.F = 8.0
        self.N = 40
        self.dt = 0.05
        self.u = np.full(self.N, self.F) + np.random.rand(self.N)

    def L96(self, u):
        F, N = self.F, self.N
        f = np.zeros(N)
        for k in range(2, N-1):
            f[k] = (u[k+1] - u[k-2]) * u[k-1] - u[k] + F
        f[0] = (u[1] - u[N-2]) * u[N-1] - u[0] + F
        f[1] = (u[2] - u[N-1]) * u[0] - u[1] + F
        f[N-1] = (u[0] - u[N-3]) * u[N-2] - u[N-1] + F
        return f

    def Model(self, u):
        dt = self.dt
        s1 = self.L96(u)
        s2 = self.L96(u + s1 * dt / 2)
        s3 = self.L96(u + s2 * dt / 2)
        s4 = self.L96(u + s3 * dt)
        du = u + (s1 + 2 * s2 + 2 * s3 + s4) * (dt / 6)
        return du

    def Data_generate(self, Time_Step=1460):
        t_data, o_data = [], []
        u = self.u

        for _ in range(Time_Step):
            u = self.Model(u)

        for i in range(Time_Step):
            u = self.Model(u)
            t_data.append(u)
        df_t = pd.DataFrame(t_data)
        df_t.to_csv('t_data.csv')
        t_data = np.array(t_data)

        for i in range(Time_Step):
            noisy_observation = t_data[i, :] + np.random.randn(self.N)
            o_data.append(noisy_observation)
        df_o = pd.DataFrame(o_data)
        df_o.to_csv('o_data.csv')
    

In [8]:
generate = Datagenerater()
generate.Data_generate()